In [ ]:
import pandas as pd

In [ ]:
# 將機場名稱中的Int'l字樣去除，並去除前後空白
df_corp['departure_airport'] = df_corp['departure_airport'].str.replace("Int\'l", "").str.replace("Intl", "").str.strip()
df_corp['arrival_airport'] = df_corp['arrival_airport'].str.replace("Int\'l", "").str.replace("Intl", "").str.strip()